# Predicting Surival on the Titanic: Let's do Data Science

Let's dive into some predictive modeling. Trust me, it's useful. (data source: Kaggle.com)

So our goal is to predict whether a given passenger survived the maiden voyage of the Titanic way back in ninteen-something-teen. Is this a practical thing to know? Not really, unless you have a time machine. But it is a good way to experiment with some data science. In this notebook, I'll be going over the easier method (thank god), using a RandomForestClassifier instead of going through all the math of a decision tree step by step. If you're a masochist, or a math-person (same thing basically), you can check out my python script creating a decision tree from scratch.

First things first, we need tools. 



In [119]:
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
import numpy as np
import pandas as pd
import pprint

# Cleaning Up the Dataset

Next let's take a look at our dataset. Data is from the real world, and as a result is often messy. If you filled your car up with raw oil striaght from the ground, you'd destroy your engine. Let's refine our data so our model doesn't spit fumes at us when we try to start it up. 

In [343]:
df = pd.read_csv('train.csv')
print "Number of Passengers Total: ", len(df)

Number of Passengers Total:  891


In [171]:
df.head(10)
#show us the first 10 rows

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Ok, right from the start there are a few things we need to deal with. First is the 'Nan' values in the 'Cabin', and 'Age' columns. Any 'Nan' values we'll need to decide what we're going to do with case-by-case. Also, there are some categorical variables we'll want to change too, such as 'Sex' and 'Pclass'. First the 'Nans', since those are more obnoxious to deal with.




In [344]:
#count the Nan's in Age
df1 = df[df['Age'].isnull()]
print 'Number of Nan in Age: ',len(df1)

Number of Nan in Age:  177


In [345]:
#count the Nan's in Cabin
df1 = df[df['Cabin'].isnull()]
print 'Number of Nan in Cabin: ',len(df1)

Number of Nan in Cabin:  687


In [346]:
#count the Nan's in Embarked
df1 = df[df['Embarked'].isnull()]
print 'Number of Nan in Embarked: ',len(df1)

Number of Nan in Embarked:  2


In [41]:
#Fill missing values with Median
median_age = df['Age'].median()
df['Age'] = df['Age'].fillna(median_age)

In [203]:
#Set Nan's as another Class
df['Cabin'] = df['Cabin'].fillna('9001')

In [204]:
#Drop Nans
df = df.dropna(subset=['Embarked'])

Notice the first thing I did was see where the missing data was. Only two missing values in 'Embarked' so I feel fine about getting rid of those two without affecting my predictions. For 'Age' and 'Cabin' though, HUGE chunks of my data are Nan values. For 'Age', I replaced all the Nan values with the median age of the entire ship so the nan values would not impact the model too much. For 'Cabin', I gave the nan-values a new classification (9001). I'm not surprized most passengers didn't have a cabin. Most folks were in steerage with a bunkbed if they were lucky. Now our model will acount for those folk.

Next thing I need to do is take all those pesky classification variables like 'Embarked' or 'Sex' and turn them into numbers the model can read. For sex, all I'm going to do is assign Females a value of 1, and males a value of 0. For 'Pclass' and others, we have more than 2 options. We'll create some dummy variables below so that we have a 1 written when a certain variable is true.

In [205]:
df1 = pd.get_dummies(df, columns=['Pclass', 'SibSp', 
                                 'Parch', 'Embarked'], drop_first = True)

In [206]:
b_loon = {'male': 0, 'female':1}
df1['Sex'] = df1['Sex'].map(b_loon)

You might notice that the variables with the Parents/Children and Siblings/Spouses I included in my categorical split here. While my model could take those variables as numeric, I'm interested to see how each ratio for either variable effects your chances of survival. My total number of variables now is 24, which is not too many. 

What about Cabin? Well, the interesting thing about Cabin is the section each cabin is in ('A','B', and so on). Let's pull that part of the data out, and make more dummy variables for each section. 



In [207]:
cabin_sections = []
for x in df1['Cabin']:
    cabin_sections.append(x[0])
df1['Cabin'] = cabin_sections

In [208]:
df1 = pd.get_dummies(df1, columns=['Cabin'], drop_first=True)
df1[['Cabin_A','Cabin_B']] #and so on

,Cabin_A,Cabin_B
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


Now we're only looking at the cabins by section, not caring about which number they are. If we find our model is not as accurate as we'd like, we can come back here and re-work this a bit.

Now, while the data is a lot less pretty to us humans, it's looking very good for the model who likes numbers rather than readability. But wait, we've still got two variables here we haven't accounted for yet, namely 'Name' and 'Ticket'. While we could take these variables and try to use them in our model with some feature engineering, for now we'll operate under the (frankly compelling) argument that your ticket number and your name will not affect your chances of survival. 

Here is our dataset now!

In [209]:
del df1['Name']
del df1['Ticket']
df = df1
df

,PassengerId,Survived,Name,Sex,Age,Fare,Pclass_2,Pclass_3,SibSp_1,SibSp_2,...,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T
0,1,0,"Braund, Mr. Owen Harris",0,22.0,7.2500,0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,71.2833,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,3,1,"Heikkinen, Miss. Laina",1,26.0,7.9250,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,53.1000,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0
4,5,0,"Allen, Mr. William Henry",0,35.0,8.0500,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
5,6,0,"Moran, Mr. James",0,NaN,8.4583,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
6,7,0,"McCarthy, Mr. Timothy J",0,54.0,51.8625,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
7,8,0,"Palsson, Master. Gosta Leonard",0,2.0,21.0750,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
8,9,1,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",1,27.0,11.1333,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
9,10,1,"Nasser, Mrs. Nicholas (Adele Achem)",1,14.0,30.0708,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Building a Model

Now that the boring annoying part is done, let's build a model! 

First thing we'll need to do is split up our data into the 'predictors' and the 'predictions'. Our 'Survived' column is what we care about predicting, so that is our y data. everything else is our X data.

In [15]:
y = df.pop('Survived')
X = df.values

Now let's get ready for some cross-validation. 

If a fortune teller says she can predict your future, how do you test her claim? You ask to predict something, and you wait to see if it came true. For us, we can't wait for the titanic to sink again (again, unless we had a time-machine). So how do we test our predictions out? We do something called a train_test_split, which separates our data into two parts. One part specifically meant to train the model, the other to test how well we did with new data the model hasn't seen yet. Whene we test the model, we're seeing how well we would do on new data, or 'out-of-sample' data. Building a model is no use unless you can prove it works, right?

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

Now here comes the part we've all been waiting for. Building the model to predict our fates!

Sharpen your pencils because here comes the math! Let's get busy!

In [17]:
rf1 = RandomForestClassifier(criterion='entropy', max_features= 'sqrt', n_estimators= 400)
rf1 = rf1.fit(X_train,y_train)

Wait, that's it? That was two lines of code! What about my masters-PHD-knighthood in applied-theoretical-statistical-mathematic-computer-science-theory? 

The cool thing about the easy way is that it's easy because the tools we got in the first cell did the math for us! There is SO much math going on in the cell above, it is insane in the membrane.

If you care to learn the concept of how a random forest (and the decision trees that they consist of), here's a good article to check out:

https://dimensionless.in/introduction-to-random-forest/

In [34]:
predicted_y = rf1.predict(X_test)

So now we have our predicted values of our data that we set aside just to test ourselves out. Let's see how we did by comparing it to our actual y_values in our test set. I'm going to do this part the long way to walk you guys through what we're doing.

In [35]:
def scoring(predicted_y,y_test):
    '''
    Providing a score of how well our predictions match up with our actual real values of survived vs. died.
    
    Input:
    Predicted_y: our predicted values of each row (survived or died represented as 1 or 0 respectively)
    y_test: the actual values'''
    total_right = 0.0
    for prediction,outcome in zip(predicted_y,y_test):
        if prediction == outcome:
            total_right+=1
    return 'Score:', total_right/len(y_test)
scoring(predicted_y,y_test)

('Score:', 0.8052434456928839)

Okay!

So that's about 80% of the time we are accurately predicting whether someone onboard the Titanic lived or died. In some cases, this could be all we needed to do and we can wipe our hands of any more work here. 

But if you're like me, you're not satisfied with an 81%. What can we do to get that score up? Remember that line of code I said was doing a ton of math for us? We passed in a bunch of parameters like 'max_features' and junk that we all didn't care about cause our life was easy. Well, one thing we could do to try and boost that score is play around with those parameters until our score gets higher. It's a bit like pushing random buttons on a space ship. Eventually you'll hit the hyperdrive and go into lightspeed, but we need to try them all first. 

But there are tons of combinations of possible parameter settings! Do we go through them all and write our scores on a piece of paper?

Remember those tools from the first cell? We can use those to search through a ton of options for us! Yay easy way!

In [20]:
param_grid = {'n_estimators': [100,400,500,1000],
               'criterion' : ['entropy'],
               'max_features' : ['auto', 'sqrt'],
              'min_samples_split': [2, 4],
             'max_depth': [None,5,10,20]}
gs_cv = GridSearchCV(rf1, 
                     param_grid, 
                     scoring='accuracy', 
                     n_jobs = -1, 
                     verbose=5).fit(X_train, y_train)

gs_cv.best_params_, gs_cv.best_score_, gs_cv.best_estimator_


Fitting 3 folds for each of 64 candidates, totalling 192 fits
[CV] max_features=auto, min_samples_split=2, criterion=entropy, max_depth=None, n_estimators=100 
[CV] max_features=auto, min_samples_split=2, criterion=entropy, max_depth=None, n_estimators=100 
[CV] max_features=auto, min_samples_split=2, criterion=entropy, max_depth=None, n_estimators=100 
[CV] max_features=auto, min_samples_split=2, criterion=entropy, max_depth=None, n_estimators=400 
[CV]  max_features=auto, min_samples_split=2, criterion=entropy, max_depth=None, n_estimators=100, score=0.822115, total=   1.4s
[CV]  max_features=auto, min_samples_split=2, criterion=entropy, max_depth=None, n_estimators=100, score=0.821256, total=   1.4s
[CV] max_features=auto, min_samples_split=2, criterion=entropy, max_depth=None, n_estimators=400 
[CV] max_features=auto, min_samples_split=2, criterion=entropy, max_depth=None, n_estimators=400 
[CV]  max_features=auto, min_samples_split=2, criterion=entropy, max_depth=None, n_estimator

[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   13.8s


[CV]  max_features=auto, min_samples_split=4, criterion=entropy, max_depth=None, n_estimators=100, score=0.811594, total=   1.2s
[CV] max_features=auto, min_samples_split=4, criterion=entropy, max_depth=None, n_estimators=100 
[CV]  max_features=auto, min_samples_split=4, criterion=entropy, max_depth=None, n_estimators=100, score=0.830918, total=   1.2s
[CV] max_features=auto, min_samples_split=4, criterion=entropy, max_depth=None, n_estimators=400 
[CV]  max_features=auto, min_samples_split=2, criterion=entropy, max_depth=None, n_estimators=1000, score=0.817308, total=  10.2s
[CV] max_features=auto, min_samples_split=4, criterion=entropy, max_depth=None, n_estimators=400 
[CV]  max_features=auto, min_samples_split=2, criterion=entropy, max_depth=None, n_estimators=1000, score=0.821256, total=  10.2s
[CV] max_features=auto, min_samples_split=4, criterion=entropy, max_depth=None, n_estimators=400 
[CV]  max_features=auto, min_samples_split=4, criterion=entropy, max_depth=None, n_estimat

[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  2.0min


[CV]  max_features=auto, min_samples_split=4, criterion=entropy, max_depth=5, n_estimators=400, score=0.840580, total=   4.7s
[CV] max_features=auto, min_samples_split=4, criterion=entropy, max_depth=5, n_estimators=1000 
[CV]  max_features=auto, min_samples_split=4, criterion=entropy, max_depth=5, n_estimators=500, score=0.826923, total=   5.7s
[CV] max_features=auto, min_samples_split=4, criterion=entropy, max_depth=5, n_estimators=1000 
[CV]  max_features=auto, min_samples_split=4, criterion=entropy, max_depth=5, n_estimators=500, score=0.797101, total=   5.7s
[CV] max_features=auto, min_samples_split=4, criterion=entropy, max_depth=5, n_estimators=1000 
[CV]  max_features=auto, min_samples_split=4, criterion=entropy, max_depth=5, n_estimators=500, score=0.840580, total=   5.7s
[CV] max_features=sqrt, min_samples_split=2, criterion=entropy, max_depth=5, n_estimators=100 
[CV]  max_features=sqrt, min_samples_split=2, criterion=entropy, max_depth=5, n_estimators=100, score=0.817308, t

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  4.6min


[CV]  max_features=auto, min_samples_split=4, criterion=entropy, max_depth=20, n_estimators=100, score=0.821256, total=   1.2s
[CV] max_features=auto, min_samples_split=4, criterion=entropy, max_depth=20, n_estimators=100 
[CV]  max_features=auto, min_samples_split=4, criterion=entropy, max_depth=20, n_estimators=100, score=0.855072, total=   1.2s
[CV] max_features=auto, min_samples_split=4, criterion=entropy, max_depth=20, n_estimators=400 
[CV]  max_features=auto, min_samples_split=2, criterion=entropy, max_depth=20, n_estimators=1000, score=0.817308, total=  12.6s
[CV] max_features=auto, min_samples_split=4, criterion=entropy, max_depth=20, n_estimators=400 
[CV]  max_features=auto, min_samples_split=4, criterion=entropy, max_depth=20, n_estimators=400, score=0.822115, total=   5.7s
[CV] max_features=auto, min_samples_split=4, criterion=entropy, max_depth=20, n_estimators=400 
[CV]  max_features=auto, min_samples_split=2, criterion=entropy, max_depth=20, n_estimators=1000, score=0.8

[Parallel(n_jobs=-1)]: Done 192 out of 192 | elapsed:  5.8min finished


({'criterion': 'entropy',
  'max_depth': 20,
  'max_features': 'sqrt',
  'min_samples_split': 4,
  'n_estimators': 100},
 0.842443729903537,
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
             max_depth=20, max_features='sqrt', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=4, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
             verbose=0, warm_start=False))

In [20]:
rf2 = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
             max_depth=20, max_features='sqrt', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
             verbose=0, warm_start=False)


In [36]:
rf2.fit(X_train,y_train)
predicted_y = rf2.predict(X_test)
scoring(predicted_y,y_test)

('Score:', 0.8164794007490637)

That probably took a while. That's a lot of math you're asking from your computer. Thank it when it's done.

So we've been able to find a model who's score is about 1% higher than before. While that might not sound like much, remember we're dealing with life and death here. That extra 1% would look very nice to someone trying to decide whether or not to board the next Titanic. 

While 81% is a good score here, what can we do to boost our score even higher? Well, there are different methods we could try instead of random forests. The Boosting method often can achieve higher scores, but it takes much more time to compute and finding the right parameters to tune the model can be a bit like trying to find a lightswitch in the dark. Is there any way we can boost our accuracy without changing our model? Yes.




# Fixing Age relative to Name

Remember when I said that a person's name did not mater when it comes to survival? It does make sense, after all iceburgs do not care what your family name is, they'll sink your boat either way. However, there are things we can tell about you from your name. For example, if you have the phrase 'Master' in your name, back in the early 20th century that meant you were younger than 12 years old. Remember how we filled in the missing values in the 'Age' column with the median age of 28 years old? Well, we know intuitively that anyone with the word 'Master' in their name won't be 28 years old! More over, if you were a Miss instead of Mrs, you were more likely to be on the younger side as well. 

Let's go back to our DataFrame and change the missing values of age to make more sense depending on someone's name. 

In [432]:
#Reload the dataset
df = pd.read_csv('train.csv')
df['Cabin'] = df['Cabin'].fillna('9001')
df = df.dropna(subset=['Embarked'])
df = pd.get_dummies(df, columns=['Pclass', 'SibSp','Parch', 'Embarked'], drop_first = True)
b_loon = {'male': 0, 'female':1}
df['Sex'] = df['Sex'].map(b_loon)
# cabin_sections = []
# for x in df['Cabin']:
#     cabin_sections.append(x[0])
# df['Cabin'] = cabin_sections
# df = pd.get_dummies(df, columns=['Cabin'], drop_first=True)
del df['Cabin']
del df['Ticket']

In [433]:
#Median age for anyone with 'Master' in their name.
masters = df[df['Name'].str.contains('Master.')==True]
masters_median_age = masters['Age'].median()
print 'Median age of Masters: ', masters_median_age

Median age of Masters:  3.5


In [434]:
#Median age for anyone with 'Miss' in their name.
miss = df[df['Name'].str.contains('Miss.')==True]
miss_median_age = miss['Age'].median()
miss_median_age

21.0

In [435]:
searchfor = ['Miss.','Master.']
other = df[df['Name'].str.contains('|'.join(searchfor))==False]
other_median_age = other['Age'].median()
other_median_age

31.0

In [436]:
df1 = df.copy()

In [437]:
df1['Age'][(df1['Age'].isnull()) & (df1['Name'].str.contains('Master.'))] = masters_median_age
df1['Age'][(df1['Age'].isnull()) & (df1['Name'].str.contains('Miss.'))] = miss_median_age
df1['Age'][(df1['Age'].isnull())] = other_median_age

/Users/William/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/William/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/William/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [438]:
del df1['Name']
df_ages_fixed = df1

OK! So now we have our ages that make sense given the passengers name. The 'Masters' are younger, the 'Miss's are a little older, and everyone else's median age has risen as a result to 31 years old instead of 28. Now let's plug this newer better dataset into our model and see how we do!

In [451]:
df1.head()

,PassengerId,Sex,Age,Fare,Pclass_2,Pclass_3,SibSp_1,SibSp_2,SibSp_3,SibSp_4,...,SibSp_8,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Parch_9,Embarked_Q,Embarked_S
0,892,0,34.5,7.8292,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,893,1,47.0,7.0000,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,894,0,62.0,9.6875,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,895,0,27.0,8.6625,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,896,1,22.0,12.2875,0,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [440]:
y = df_ages_fixed.pop('Survived')
X = df_ages_fixed.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

rf2 = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
             max_depth=20, max_features='sqrt', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
             verbose=0, warm_start=False)


In [441]:
rf2.fit(X_train,y_train)
predicted_y = rf2.predict(X_test)
scoring(predicted_y,y_test)


('Score:', 0.8239700374531835)

Look at that! We're now at 83-85% accuracy after pulling some information about names to better estimate ages. 

Now, could we look for other ways to boost our score? Sure, but we need to think about just how valuable our time is and whether that extra few percent we might be able to squeeze out could be worth the effort. If this is a competition where the winner with the highest accuracy wins a million dollars, then yeah let's go nuts. If this is a marketing project and we need to predict which customers are going to churn by the end of the day or else our boss is going to fire us, we should stop here. Try not to get too bogged down with hunting accuracy and instead think about the big picture. Have we accopmlished what we intended? What else can we gleam from this? 

Remember, this kind of model is only usefull if we can convince others about it. Make it too complicated, and it becomes difficult to explain. Make it too simple, and it won't be as accurate. Keep the original problem in mind as you go, and you won't waste your time. 